In [196]:
# coding: utf-8
import requests,re
from bs4 import BeautifulSoup
import pandas as pd


In [187]:
ListShazam = pd.read_csv('shazam1.csv',sep=',')
ListShazam['Paroles'] = "vide"
ListShazam['MotsFrequents'] = "vide"

In [197]:
# youtube
def importSoup(url):
    request_headers = {
       "Accept-Language": "en-US,en;q=0.5",
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
       "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
       "Referer": "http://thewgfdgebsite.com",
       "Connection": "keep-alive"
    }
    res = requests.get(url, headers = request_headers)
    if res.status_code == 200:
        html_doc =  res.text
        soup = BeautifulSoup(html_doc,"html.parser")
        return soup



In [ ]:
#NbVues = soup.find_all("div",attrs={'class':'genres'})

In [198]:

url = r"https://search.azlyrics.com/search.php?q=Bruno+Mars+That%27s+What+I+Like+"
soup = importSoup(url)

In [199]:
for i in range(len(ListShazam)):
    soup = soupParoles = all_links = Paroles = parole = urlParoles = ""
    Title, Artist = ListShazam["Title"][i], ListShazam["Artiste"][i]
    Titlep = Title.replace(' ','+')
    Artistp = Artist.replace(' ','+')
    url = r"https://search.azlyrics.com/search.php?q=" + Titlep + '+' + Artistp
    soup = importSoup(url)
    all_links = map(lambda x : x.attrs['href'] , soup.find_all("a"))
    for a in all_links:
        if re.search(r'https://www.azlyrics.com/lyrics/', a):
            urlParoles = a
            break
    if urlParoles != "":
        soupParoles = importSoup(urlParoles)
        Paroles = soupParoles.find_all("div",attrs={'class':'col-xs-12 col-lg-8 text-center'})
        parole = re.sub('\n',' ',Paroles[0].findAll('div')[6].text)
        parole = re.sub('\r',' ',parole)
        wordcount = Counter(parole.split(' '))
        wc = wordcount.most_common()
        ListShazam['Paroles'][i] = parole
        ListShazam['MotsFrequents'][i] = wc[0][0]
    else:
        ListShazam['Paroles'][i] = "vide"


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

NameError: name 'paroles' is not defined

In [132]:
re.sub('\n',' ',Paroles[0].findAll('div')[6].text)

" \r What the hell do you mean I've got to wait in line? This is comlpete and total bullshit! If if I show you my tits Would you let me in? Can you just Just take another drink with me? Thanks! Later Bitches!  Oh my god! This is like This is like the best song ever! I just I just love it so much! Shit! I don't feel so good... I shouldn't have that last shot! Later Bitches!  Oh my god! I'm so horny right now... I shouldn't have said that outloud I'm not like wearing any underwear... So, you know... Can I have another shot? Hey, before we get out of here Can you... Can you just do me one favor? Can you say... Later Bitches! "

[('you', 7),
 ('I', 7),
 ('', 6),
 ('Can', 5),
 ('just', 4),
 ('This', 3),
 ('is', 3),
 ('my', 3),
 ('Later', 3),
 ('Bitches!', 3),
 ('like', 3),
 ('so', 3),
 ('have', 3),
 ('the', 2),
 ('do', 2),
 ('me', 2),
 ('another', 2),
 ('Oh', 2),
 ('god!', 2),
 ("shouldn't", 2),
 ('that', 2),
 ("I'm", 2),
 ('What', 1),
 ('hell', 1),
 ('mean', 1),
 ("I've", 1),
 ('got', 1),
 ('to', 1),
 ('wait', 1),
 ('in', 1),
 ('line?', 1),
 ('comlpete', 1),
 ('and', 1),
 ('total', 1),
 ('bullshit!', 1),
 ('If', 1),
 ('if', 1),
 ('show', 1),
 ('tits', 1),
 ('Would', 1),
 ('let', 1),
 ('in?', 1),
 ('Just', 1),
 ('take', 1),
 ('drink', 1),
 ('with', 1),
 ('me?', 1),
 ('Thanks!', 1),
 ('best', 1),
 ('song', 1),
 ('ever!', 1),
 ('love', 1),
 ('it', 1),
 ('much!', 1),
 ('Shit!', 1),
 ("don't", 1),
 ('feel', 1),
 ('good...', 1),
 ('last', 1),
 ('shot!', 1),
 ('horny', 1),
 ('right', 1),
 ('now...', 1),
 ('said', 1),
 ('outloud', 1),
 ('not', 1),
 ('wearing', 1),
 ('any', 1),
 ('underwear...', 1),
 ('So,', 1),
 ('kno

In [178]:
for item in sor.items(): print("{}\t{}".format(*item))

AttributeError: 'list' object has no attribute 'items'

In [166]:
textparole = ListShazam['Paroles'][0]

0     vide
1     vide
2     vide
3     vide
4     vide
5     vide
6     vide
7     vide
8     vide
9     vide
10    vide
11    vide
12    vide
13    vide
14    vide
15    vide
16    vide
17    vide
18    vide
19    vide
20    vide
21    vide
22    vide
23    vide
24    vide
25    vide
26    vide
27    vide
28    vide
29    vide
      ... 
70    vide
71    vide
72    vide
73    vide
74    vide
75    vide
76    vide
77    vide
78    vide
79    vide
80    vide
81    vide
82    vide
83    vide
84    vide
85    vide
86    vide
87    vide
88    vide
89    vide
90    vide
91    vide
92    vide
93    vide
94    vide
95    vide
96    vide
97    vide
98    vide
99    vide
Name: Paroles, Length: 100, dtype: object